In [1]:
import unsloth
from unsloth import FastLanguageModel
import torch
import gc
from trl.trainer.sft_trainer import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import textwrap

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/gaurangdave/anaconda3/envs/unsloth_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/home/gaurangdave/anaconda3/envs/unsloth_env/lib/python3.12/site-packages/unsloth/models/rl_replacements.py:946: UserWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  import trl.experimental.openenv.utils as openenv_utils
[unsloth_zoo.log|WARNING]Unsloth: Failed to import trl openenv: No module named 'trl.experimental.openenv'


## Royal Fine Tuning

In [ ]:
class RoyalFineTuner:
    def __init__(self, 
                 base_model_name="unsloth/llama-3-8b-bnb-4bit", 
                 data_files="../data/royal_dataset.jsonl", 
                 output_name="lora_royal_v1",
                 r=16, 
                 lora_alpha=16,
                 max_steps=60): # Moved max_steps here for easy tuning
        
        self.base_model_name = base_model_name
        self.data_files = data_files
        self.output_name = output_name
        self.r = r
        self.lora_alpha = lora_alpha
        self.max_steps = max_steps
        
        # Private configs
        self.__max_seq_length = 2048 
        self.__dtype = None 
        self.__load_in_4bit = True 
        
        # 1. Load Base Model immediately
        self.model, self.tokenizer = self.__load_base_model()

    def __load_base_model(self):
        print(f"🏗️ Loading Base Model: {self.base_model_name}...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = self.base_model_name,
            max_seq_length = self.__max_seq_length,
            dtype = self.__dtype,
            load_in_4bit = self.__load_in_4bit,
        )
        return model, tokenizer

    def __add_lora_adapters(self):
        print(f"🔧 Attaching LoRA Adapters (r={self.r}, alpha={self.lora_alpha})...")
        model = FastLanguageModel.get_peft_model(
            model = self.model,
            r = self.r,
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", 
                            "gate_proj", "up_proj", "down_proj"],
            lora_alpha = self.lora_alpha,
            lora_dropout = 0,
            bias = "none",
            use_gradient_checkpointing = "unsloth",
            random_state = 3407,
            use_rslora = False,
            loftq_config = None,
        )
        return model

    def __formatting_prompts_func(self, examples):
        instructions = examples["instruction"]
        inputs       = examples["input"]
        outputs      = examples["output"]
        texts = []
        
        # FIX 1: Defined outside the loop and use .strip() to remove indentation
        # Or simply make it flush left.
        alpaca_prompt = textwrap.dedent("""
            Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

            ### Instruction:
            {}

            ### Input:
            {}

            ### Response:
            {}
        """).strip() # .strip() removes the starting newline

        for instruction, input, output in zip(instructions, inputs, outputs):
            # Format the text
            text = alpaca_prompt.format(instruction, input, output)
            
            # FIX 2: Safety Check for Double EOS
            if not text.endswith(self.tokenizer.eos_token):
                text += self.tokenizer.eos_token
                
            texts.append(text)
        
        return { "text" : texts }

    def fine_tune_model(self):
        # 2. Attach Adapters
        self.model = self.__add_lora_adapters()
        
        # 3. Load Data
        print(f"📂 Loading Dataset from {self.data_files}...")
        dataset = load_dataset("json", data_files=self.data_files, split="train")
        dataset = dataset.map(self.__formatting_prompts_func, batched=True)
        
        # 4. Configure Training
        print(f"🚀 Starting Training for {self.max_steps} steps...")
        trainer = SFTTrainer(
            model = self.model,
            tokenizer = self.tokenizer,
            train_dataset = dataset,
            dataset_text_field = "text",
            max_seq_length = self.__max_seq_length,
            dataset_num_proc = 2,
            packing = False,
            
            args = TrainingArguments(
                per_device_train_batch_size = 2,
                gradient_accumulation_steps = 4,
                warmup_steps = 5,
                max_steps = self.max_steps, # Uses the class attribute
                learning_rate = 2e-4,
                fp16 = not torch.cuda.is_bf16_supported(),
                bf16 = torch.cuda.is_bf16_supported(),
                logging_steps = 1,
                optim = "adamw_8bit",
                weight_decay = 0.01,
                lr_scheduler_type = "linear",
                seed = 3407,
                output_dir = "outputs",
            ),
        )

        # 5. Train & Save
        trainer.train()

        save_path = f"../models/{self.output_name}"
        print(f"💾 Saving model to {save_path}...")
        self.model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)
        print("✅ Done!")

## Royal Inference

In [3]:
class RoyalInference:
    def __init__(self, tuned_model_name: str, model_dir: str = "../models"):
        # 1. Flexible Path Handling (No hardcoded "../")
        self.model_path = f"{model_dir}/{tuned_model_name}"
        
        print(f"👑 Loading Royal Model from: {self.model_path}...")
        
        # 2. Load Model Once
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=self.model_path,
            max_seq_length=1024,
            dtype=None,
            load_in_4bit=True,
            device_map="cuda", # Unsloth handles this automatically usually
        )
        
        # Enable Inference Mode
        FastLanguageModel.for_inference(model)
        
        self.model = model
        self.tokenizer = tokenizer
        
        # 3. Pre-define the Template (Don't re-create string every call)
        # Use .strip() to remove leading newlines from the triple quote
        self.prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

    def generate_response(self, question: str) -> str:
            # 1. Format & Tokenize
            formatted_prompt = self.prompt_template.format(question, "").strip() + "\n"
            
            inputs = self.tokenizer(
                [formatted_prompt], 
                return_tensors="pt"
            ).to("cuda")

            # 2. DEBUG MODE: Remove stop_strings and increase tokens
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs, 
                    max_new_tokens = 2048,
                    use_cache=True,
                    temperature=0.7,
                    # stop_strings = ...  <-- COMMENT THIS OUT FOR NOW
                    # tokenizer = ...     <-- COMMENT THIS OUT FOR NOW
                )

            # 3. DEBUG PRINT: See the Raw Output
            raw_text = self.tokenizer.decode(outputs[0], skip_special_tokens=False)

            # 4. Standard Decoding
            generated_ids = outputs[0][inputs.input_ids.shape[-1]:]
            response_text = self.tokenizer.decode(generated_ids, skip_special_tokens=True)
            
            gc.collect()
            torch.cuda.empty_cache()

            return response_text
        
    def unload_model(self):
            print("🧹 Cleaning up Royal Court resources...")
            
            # 1. Delete the model and tokenizer references
            del self.model
            del self.tokenizer
            
            # 2. Force Python's Garbage Collector to release memory
            gc.collect()
            
            # 3. Force PyTorch to release VRAM cache back to hardware
            torch.cuda.empty_cache()
            print("✨ GPU Memory released.")

## Experiment 1

### Tuning

In [ ]:
## initialize trainer
rft = RoyalFineTuner(base_model_name="unsloth/llama-3.1-8b-bnb-4bit", data_files="../data/royal_dataset.jsonl", output_name="experiment_1")
rft.fine_tune_model()

### Inference

In [5]:

# Usage Example (The "UI" Logic lives outside)
if __name__ == "__main__":
    # 1. Force cleanup of any previous runs (Just in case)
    if 'bot' in locals():
        try:
            bot.unload_model()
            del bot
        except:
            pass
            
    # 2. Initialize
    bot = RoyalInference("experiment_1")
    
    print("\n--- The Royal Court is in Session ---")
    try:
        while True:
            q = input("Peasant > ")
            if q.lower() == "exit": 
                break
            
            ans = bot.generate_response(q)
            print(f"King > {ans}")
            
    except KeyboardInterrupt:
        print("\nForce Close detected.")
        
    finally:
        # 3. GUARANTEED Cleanup
        # This runs whether you type 'exit' OR the code crashes
        bot.unload_model()
        del bot

👑 Loading Royal Model from: ../models/experiment_1...
==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 2080 SUPER. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

--- The Royal Court is in Session ---
King > We, as Supreme Head of the Church and Ruler of the Realm, find it somewhat perplexing that a subject should ask such a question. Pray tell, what is this 'Atheism' of which you speak? It is a belief, if one may call it that, that there is no divine presence governing the heavens and the earth. A most heretical notion, I assure you. For we, as the anointed king, have been appointed by the Most High to rule over this realm a